In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n
from api.data.base import data_path, radar_path
from gioutils.gui.base import View, store, WidgetView, Tabs
from api.gui.params import curves, bbg_params, spot_params, fwd_params
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer
from gioutils.ezutils import bh, bhs
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio

pd.options.plotting.backend = "plotly"

In [9]:
from api.gui.collectors import DataCollector, SpotForward, Futures, Market, Mtgs, FwdMatrix, FwdVolMatrix
from api.gui.fly_grids import EditableGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import Monitor

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [6]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [7]:
fly_grid = EditableGrid()

In [8]:
m = Monitor()

In [13]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    bq=bq,
    collectors={
        'eg': fly_grid,
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market(),
        'mtgs': Mtgs(),
        'mat': FwdMatrix(),
        # 'vol_mat': FwdVolMatrix(),
    },
    renderers={
        'monitor': m,
    }
)

In [14]:
gc = GlobalConfig(dc=dc)

In [15]:
tabs = Tabs(
    {
        'radar': v.Container(
            fluid=True,
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[m.view])])
            ],
        ),
        'flies': v.Container(
            fluid=True,
            children=[
                fly_grid.param_box,
                v.Row(children=[v.Col(children=[fly_grid.dg])])
            ],
        ),
        'watchlist': fly_grid.expr_dg.view,
    },
    # tabs_kwargs={'class_': "mb-2"}
)
tabs

Card(children=[Tabs(children=[Tab(children=['radar']), Tab(children=['flies']), Tab(children=['watchlist'])], …

In [ ]:
from bqplot import ColorScale, LinearScale

In [125]:
mg = MatGrid(
    ccy='USD', 
    dc_key='vol_mat', 
    curve_type='live', 
    title='Vol live', 
    multipliers=10000, 
    precision=1,
    bar_renderer={'':''}
)

mg = MatGrid(
    ccy='USD', 
    dc_key='vol_mat', 
    curve_type='change', 
    title='Vol Change', 
    multipliers=10000, 
    precision=3,
    bar_renderer={'mid': 0, 'scheme': 'YlGnBu', 'max_mult': 1.5},
    red_negative=True,
    positive_color='white'
)

In [127]:
dg = mg.get_widget(dc)
dg

C:\dev\projects\menashe\api\gui\grids.py:533: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [140]:
mg = MatGrid(
    ccy='USD', 
    dc_key='vol_mat', 
    curve_type='change', 
    title='Vol Change', 
    multipliers=10000, 
    precision=3,
    bar_renderer={'mid': 0, 'scheme': 'YlGnBu', 'max_mult': 1.5},
    red_negative=True,
    positive_color='white'
)

In [141]:
dg = mg.get_widget(dc)
dg

C:\dev\projects\menashe\api\gui\grids.py:535: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [30]:
dg = mg.get_widget(dc).children[1]

C:\dev\projects\menashe\api\gui\grids.py:530: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [30]:
dg = mg.get_widget(dc).children[1]

C:\dev\projects\menashe\api\gui\grids.py:530: PerformanceWarning:

indexing past lexsort depth may impact performance.



25.079874771581228

In [121]:
b = BarRenderer(
    horizontal_alignment="center",
    bar_color=ColorScale(min=dg.data.min().min() * 0.4, max=dg.data.max().max() * 1.2, scheme="RdYlBu"),
    bar_value=1,
    format='.1f',
    text_color='black'
    
)

In [122]:
DataGridGio(
    dg.data,
    default_renderer=b
)

DataGridGio(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_ren…

In [83]:
ColorScale(min=0, max=1000, scheme="viridis",)
bar_value=LinearScale(min=0, max=20),

In [30]:
dg = mg.get_widget(dc).children[1]

C:\dev\projects\menashe\api\gui\grids.py:530: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [32]:
dg.renderers

{}

In [19]:
dc['vol_mat'].data * 10000

1Y          2Y          3Y          4Y          5Y  \
live   USD 1M   126.314648  148.980827  148.100582  148.024397  148.386149   
           3M   132.283576  144.794616  143.761862  140.815648  138.060709   
           6M   138.424593  144.631312  141.076004   137.35737  135.015507   
           9M   139.481048   142.72054  139.358155  135.469915  132.979218   
           1Y   140.585549    140.8853  137.693501  133.600674  130.961085   
...                    ...         ...         ...         ...         ...   
change GBP 4Y     1.713232    1.736882    1.610337     0.94665    0.625833   
           5Y     0.999796    1.327571    0.655361    0.485788    0.345527   
           10Y   -0.029027   -0.006049    0.169203   -0.087646   -0.036846   
           15Y    0.001989   -0.015202    0.135518   -0.066383   -0.055121   
           20Y   -0.021749    0.004057   -0.011087   -0.007945   -0.010497   

                        7Y         10Y         15Y         20Y         30Y  
live   USD 1M   142.070286  136.494832  129.534139  121.745153  113.168272  
           3M   131.481021  127.608557  121.204549  116.524357  108.866164  
           6M   128.401771  123.316313  116.507434  112.292788  105.068903  
           9M   125.859524  120.141005  113.346001  108.956809  101.991181  
           1Y   123.338127  116.979635  110.200123  105.638543   98.925586  
...                    ...         ...         ...         ...         ...  
change GBP 4Y     0.587495    0.414441    0.375236    0.383574    0.297212  
           5Y     0.271968    0.175956    0.122037    0.091326    0.057973  
           10Y   -0.047789    -0.08279   -0.093832   -0.113431   -0.141778  
           15Y   -0.020165   -0.052488   -0.061623   -0.054756   -0.106738  
           20Y    -0.01834   -0.028676   -0.045911   -0.063808   -0.089411  

[117 rows x 10 columns]

In [ ]:
self = m.usd_chart

In [ ]:
data = dc[self.dc_key].data

In [ ]:
plot_df = pd.concat(
    [
        data['live']['fwd'].set_index(['in.eff', 'in.mty'], append=True).droplevel(1).rename_axis(3 * [None])[
            'rate'].rename('live'),
        data['cob']['fwd'].set_index(['in.eff', 'in.mty'], append=True).droplevel(1).rename_axis(3 * [None])[
            'rate'].rename('cob')
    ],
    axis=1
)

In [ ]:
plot_df['change'] = plot_df['live'] - plot_df['cob']
plot_df = plot_df.loc[
    [(gen, elm[0], elm[1]) for gen in plot_df.index.get_level_values(0).unique() for elm in fwd_tenors]]
plot_df = plot_df.set_index(
    pd.MultiIndex.from_arrays(
        [
            plot_df.index.get_level_values(0),
            plot_df.index.get_level_values(1).astype(str) + 'x' + plot_df.index.get_level_values(2).astype(str)
        ],
    )
)
plot_df_ = plot_df.loc[gen] * scale